In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

## Elastic Search

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 217.77it/s]


In [5]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [6]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

In [7]:
import pandas as pd

In [8]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [9]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [13]:
relevance_total

[[True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, True, False, False],
 [False, False, False, False, False],
 [False, False, False, True, False],
 [False, False, False, False, True],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [],
 [],
 [],
 [],
 [],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, True, False, False],
 [False, True, False, False, False],
 [False, True, False, False, False],
 [True

In [14]:
example = [[True, False, False, False, False], # 1
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [False, False, True, False, False], # 1
 [False, False, False, False, False], # 0
 [False, False, False, True, False], # 1
 [False, False, False, False, True], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False]] # 1

In [15]:
12 / 17

0.7058823529411765

In [17]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt += 1

    return cnt/len(relevance_total)

In [18]:
hit_rate(example)

0.7058823529411765

In [20]:
example = [[True, False, False, False, False], # 1
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [False, False, False, False, False], # 0
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [False, False, True, False, False], # 1/3
 [False, False, False, False, False], # 0
 [False, False, False, True, False], # 1/4
 [False, False, False, False, True], # 1/5
 [True, False, False, False, False], # 1
 [True, False, False, False, False], # 1
 [True, False, False, False, False]] # 1

# 1 => 1
# 2 => 1/2 = 0.5
# 3 => 1/3 = 0.333
# 4 => 1/4 = 0.25
# 5 => 1/5 = 0.2
# None = 0
# Rank = 1/Rank

In [21]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1/(rank+1)

    return total_score/len(relevance_total)

In [22]:
mrr(example)

0.5754901960784313

In [23]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7395720769397017, 0.6029788920106625)

## Min Search

In [24]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-07-19 12:42:56--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273 (4.2K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   4.17K  --.-KB/s    in 0s      

2025-07-19 12:42:57 (47.2 MB/s) - ‘minsearch.py’ saved [4273/4273]



In [25]:
import minsearch

/workspaces/LLMcourseDT/03-evaluation/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [27]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"] # added the id field
)

index.fit(documents)

In [28]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [29]:
relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:15<00:00, 292.96it/s]


In [30]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7722066133563864, 0.661454506159499)

results with Elastic Search: (0.7395720769397017, 0.6029788920106625)

### Cleaning up the code

In [32]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total)
    }

In [33]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:10<00:00, 451.43it/s]


{'hit_rate': 0.7395720769397017, 'mrr': 0.6029788920106625}

In [36]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:15<00:00, 296.35it/s]


{'hit_rate': 0.7722066133563864, 'mrr': 0.661454506159499}